<a href="https://colab.research.google.com/github/kylemcq13/Assignments/blob/master/18_5_Housing_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 18.5 Housing Assignment

* Load the **houseprices** data from Thinkful's database.
* Run your house prices model again and assess the goodness of fit of your model using F-test, R-squared, adjusted R-squared, AIC and BIC.
* Do you think your model is satisfactory? If so, why?
* In order to improve the goodness of fit of your model, try different model specifications by adding or removing some variables. 
* For each model you try, get the goodness of fit metrics and compare your models with each other. Which model is the best and why?

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

In [0]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
houses_df = pd.read_sql_query('select * from houseprices',con=engine)

engine.dispose()

In [0]:
#convert to numeric and add our categorical variables back to our dataframe

houses_df = pd.concat([houses_df,pd.get_dummies(houses_df.mszoning, prefix="mszoning", drop_first=True)], axis=1)
houses_df = pd.concat([houses_df,pd.get_dummies(houses_df.street, prefix="street", drop_first=True)], axis=1)
houses_df = pd.concat([houses_df,pd.get_dummies(houses_df.street, prefix="kitchenqual", drop_first=True)], axis=1)

cat_column_names = list(pd.get_dummies(houses_df.mszoning, prefix="mszoning", drop_first=True).columns)
cat_column_names = cat_column_names + list(pd.get_dummies(houses_df.street, prefix="street", drop_first=True).columns)
cat_column_names2 = cat_column_names + list(pd.get_dummies(houses_df.street, prefix="kitchenqual", drop_first=True).columns)

In [0]:
# Y is the target variable
Y = houses_df['saleprice']
# X is the feature set which includes
X = houses_df[['overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalbsmtsf']  + cat_column_names2]

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.769
Model:                            OLS   Adj. R-squared:                  0.767
Method:                 Least Squares   F-statistic:                     482.0
Date:                Mon, 11 Nov 2019   Prob (F-statistic):               0.00
Time:                        15:35:36   Log-Likelihood:                -17475.
No. Observations:                1460   AIC:                         3.497e+04
Df Residuals:                    1449   BIC:                         3.503e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -1.173e+05    1.8e+04     -6.502      0.000   -1.53e+05   -8.19e+04
overallqual       2.333e+04   1088.506     21.430      0.000    2.12e+04    2.55e+04
grlivarea           45.6344      2.468     18.494      0.000      40.794      50.475
garagecars        1.345e+04   2990.453      4.498      0.000    7584.056    1.93e+04
garagearea          16.4082     10.402      1.577      0.115      -3.997      36.813
totalbsmtsf         28.3816      2.931      9.684      0.000      22.633      34.131
mszoning_FV       2.509e+04   1.37e+04      1.833      0.067   -1761.679    5.19e+04
mszoning_RH       1.342e+04   1.58e+04      0.847      0.397   -1.77e+04    4.45e+04
mszoning_RL       2.857e+04   1.27e+04      2.246      0.025    3612.782    5.35e+04
mszoning_RM       9047.7894   1.29e+04      0.702      0.483   -1.62e+04    3.43e+04
street_Pave         75.5188   8145.750      0.009      0.993   -1.59e+04    1.61e+04
kitchenqual_Pave    75.5188   8145.750      0.009      0.993   -1.59e+04    1.61e+04
==============================================================================
Omnibus:                      415.883   Durbin-Watson:                   1.979
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            41281.526
Skew:                          -0.115   Prob(JB):                         0.00
Kurtosis:                      29.049   Cond. No.                     1.24e+21
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.81e-33. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

- R squared and adj R squared are at roughly 77%.
- F statistic/ pvalue are at 482.0 and 0 respective.
- AIC/BIC = 34970/35030

Roughly 23% of of the variance in the target is not explained by the model. This is better than our weather model, but there is still more room for improvement. 



To improve this model, a few things can be done. The target variable is not normally distributed, so transforming it could help. To do this, we can apply a log transform to the variable.

We can also add an interaction term here, perhaps between overallqual and totalbsmtsf. Let's try this and see if we get any improvement.

In [0]:
# create interaction term
houses_df['int_term'] = houses_df['overallqual'] * houses_df['totalbsmtsf']

# Y is the target variable
Y = np.log1p(houses_df['saleprice'])
# X is the feature set which includes
X = houses_df[['overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalbsmtsf', 'int_term']  + cat_column_names2]

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.832
Model:                            OLS   Adj. R-squared:                  0.831
Method:                 Least Squares   F-statistic:                     652.5
Date:                Mon, 11 Nov 2019   Prob (F-statistic):               0.00
Time:                        15:47:29   Log-Likelihood:                 571.34
No. Observations:                1460   AIC:                            -1119.
Df Residuals:                    1448   BIC:                            -1055.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                9.8987      0.085    115.952      0.000       9.731      10.066
overallqual          0.1627      0.007     22.792      0.000       0.149       0.177
grlivarea            0.0002   1.06e-05     19.963      0.000       0.000       0.000
garagecars           0.0860      0.013      6.659      0.000       0.061       0.111
garagearea        9.228e-05    4.5e-05      2.052      0.040    4.04e-06       0.000
totalbsmtsf          0.0004   3.71e-05     10.523      0.000       0.000       0.000
int_term         -3.999e-05   5.12e-06     -7.810      0.000      -5e-05   -2.99e-05
mszoning_FV          0.5041      0.059      8.562      0.000       0.389       0.620
mszoning_RH          0.3990      0.068      5.869      0.000       0.266       0.532
mszoning_RL          0.4974      0.055      9.102      0.000       0.390       0.605
mszoning_RM          0.3374      0.055      6.099      0.000       0.229       0.446
street_Pave          0.0019      0.035      0.054      0.957      -0.067       0.070
kitchenqual_Pave     0.0019      0.035      0.054      0.957      -0.067       0.070
==============================================================================
Omnibus:                      466.485   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4483.752
Skew:                          -1.203   Prob(JB):                         0.00
Kurtosis:                      11.241   Cond. No.                     9.60e+20
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.05e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

- R squared and adj. R squared both increased to 0.832 and 0.831 respectively.
- F statistic jumped to 652.5 and it's p value stayed at 0.
- AIC/BIC = -1119/-1055

All things considered above, our second model outperforms our original.

In [0]:
# create interaction term
houses_df['int_term'] = houses_df['overallqual'] * houses_df['totalbsmtsf']

# Y is the target variable
Y = np.log1p(houses_df['saleprice'])
# X is the feature set which includes
X = houses_df[['overallqual', 'grlivarea', 'garagecars','totalbsmtsf', 'garagearea', 'int_term', 'mszoning_RL', 'mszoning_FV']]

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.827
Model:                            OLS   Adj. R-squared:                  0.826
Method:                 Least Squares   F-statistic:                     867.9
Date:                Mon, 11 Nov 2019   Prob (F-statistic):               0.00
Time:                        15:59:26   Log-Likelihood:                 550.02
No. Observations:                1460   AIC:                            -1082.
Df Residuals:                    1451   BIC:                            -1034.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          10.2093      0.042    244.005      0.000      10.127      10.291
overallqual     0.1674      0.007     23.259      0.000       0.153       0.182
grlivarea       0.0002   1.08e-05     19.740      0.000       0.000       0.000
garagecars      0.0875      0.013      6.690      0.000       0.062       0.113
totalbsmtsf     0.0004   3.76e-05     10.594      0.000       0.000       0.000
garagearea   7.417e-05   4.54e-05      1.634      0.102   -1.49e-05       0.000
int_term    -4.132e-05   5.18e-06     -7.974      0.000   -5.15e-05   -3.12e-05
mszoning_RL     0.1693      0.012     13.688      0.000       0.145       0.194
mszoning_FV     0.1734      0.024      7.118      0.000       0.126       0.221
==============================================================================
Omnibus:                      476.206   Durbin-Watson:                   2.012
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4161.491
Skew:                          -1.264   Prob(JB):                         0.00
Kurtosis:                      10.875   Cond. No.                     7.93e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.93e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""